# VQE in DFT with PsiEmbed and Qiskit

Here we define the inputs as required by PsiEmbed. Note that we'll follow the logic of `embedding_module/run_open_shell`.

We can think of this procedure are requiring three steps:

1. Pre-embedding

    Here we define the problem, and run a low-level calculation of the whole system. From this we obtain the pre-embedded density matrices $\gamma^A$ and $\gamma^B$

    We then define the level-shift projector $P$ and embedding potential $V_{emb}$.

2. Embedding

    Using $V_{emb}$ we run a high-level method simulation of the active region to get the embedded density matrix $\gamma^A_{emb}$.

    We calculate the correction term $tr[V_{emb}(\gamma^A_{emb}-\gamma^A)]$

3. Post-embedding

    Finally we calculate the embedded energy, by removing $V_{emb}$ from the Hamiltonian, and using density matrix $\gamma^A_{emb}$.

    The total energy is then given by: $E = E[\gamma^A_{emb}] + E[\gamma^B] + g[\gamma^A, \gamma^B] + E_{nuclear} + tr[V_{emb}(\gamma^A_{emb}-\gamma^A)]$

# 0. Set Parameters

First we'll set the parameters 

In [1]:
from copy import copy
import numpy as np
from scipy import linalg
from typing import Dict

ethane = """
C      0.7516     -0.0225     -0.0209
C     -0.7516      0.0225      0.0209
H      1.1851     -0.0039      0.9875
H      1.1669      0.8330     -0.5693
H      1.1155     -0.9329     -0.5145
H     -1.1669     -0.8334      0.5687  
H     -1.1157      0.9326      0.5151
H     -1.1850      0.0044     -0.9875
    """

ethylene = """
C    3.402   0.773  -9.252
H    2.933  -0.150  -9.521
H    2.837   1.682  -9.258
C    4.697   0.791  -8.909
H    5.262  -0.118  -8.904
H    5.167   1.714  -8.641
    """

methanol = """
O     -0.6582     -0.0067      0.1730 
H     -1.1326     -0.0311     -0.6482
C      0.7031      0.0083     -0.1305
H      1.2001      0.0363      0.8431
H      0.9877      0.8943     -0.7114
H      1.0155     -0.8918     -0.6742
  """
# H     -1.1326     -0.0311     -0.6482 <--- this goes with the oxygen


formaldehyde = """
C      0.5979      0.0151      0.0688
H      1.0686     -0.1411      1.0408
H      1.2687      0.2002     -0.7717
O     -0.5960     -0.0151     -0.0686
  """

water = """
O          0.00000        0.000      0.115
H          0.00000        0.754     -0.459
H          0.00000       -0.754     -0.459
    """

h_peroxide = """
O          0.00000        0.00000        0.00000
O          1.00000        0.00000        0.00000
H          0.00000        0.50000        0.00000
H          1.00000       -0.50000        0.00000
"""

LiH = """
Li 0.0 0.0 0.0
H  0.0 0.0 1.541
"""

Li2 = """
Li 0.0 0.0 0.0
Li  0.0 0.0 2.627
"""

glycine = """
O     1.916  -0.760  -0.740 
H    -1.736  -0.252  -0.592
N    -1.476   0.232   0.252 
C    -0.012   0.296   0.348 
C     0.596  -0.652  -0.648 
O    -0.124  -1.320  -1.368 
H     0.292   0.020   1.364 
H     0.320   1.320   0.132 
H    -1.636  -0.236   1.132 
H    -1.916   1.136   0.272
  """

pyridine = """
  N      0.6816      1.1960      0.0000
  C      1.3603      0.0256      0.0000
  C      0.6971     -1.2020      0.0000
  C     -0.6944     -1.2184      0.0000
  C     -1.3895     -0.0129      0.0000
  C     -0.6712      1.1834      0.0000
  H      2.4530      0.1083      0.0000
  H      1.2665     -2.1365      0.0000
  H     -1.2365     -2.1696      0.0000
  H     -2.4837      0.0011      0.0000
  H     -1.1569      2.1657      0.0000
"""

options = {}
options['geometry'] = ethylene
options['n_active_atoms'] = 2 # number of active atoms (first n atoms in the geometry string)

run_fci = True
run_vqe = False
mos_to_use = 10

options['basis'] = 'STO-3G' # basis set 
options['low_level'] = 'B3LYP' # level of theory of the environment 
options['high_level'] = 'mp2' # level of theory of the embedded system
options['low_level_reference'] = 'rhf'
options['high_level_reference'] = 'rhf'
options['package'] = 'pyscf'
options['e_convergence'] = 1e-10

def fill_defaults(keywords) -> Dict[str, str]:
    "Fills in the default keyword values"
    # Default keywords
    default_keywords = {}
    default_keywords["package"] = "pyscf"
    default_keywords["num_threads"] = 1
    default_keywords["memory"] = 1000
    default_keywords["charge"] = 0
    default_keywords["multiplicity"] = 1
    default_keywords["low_level_reference"] = "rhf"
    default_keywords["high_level_reference"] = "rhf"
    default_keywords["partition_method"] = "spade"
    default_keywords["e_convergence"] = 1e-6
    default_keywords["d_convergence"] = 1e-6
    default_keywords["eri"] = "df"
    default_keywords["ints_tolerance"] = 1e-10
    default_keywords["driver_output"] = "output.dat"
    default_keywords["embedding_output"] = "embedding.log"
    default_keywords["operator"] = "F"
    default_keywords["level_shift"] = 1.0e6
    default_keywords["low_level_damping_percentage"] = 0
    default_keywords["high_level_damping_percentage"] = 0
    default_keywords["low_level_soscf"] = "False"
    default_keywords["high_level_soscf"] = "False"
    default_keywords["molden"] = False
    default_keywords["print_level"] = 1
    default_keywords["cc_type"] = "df"
    default_keywords["save_embedding_potential"] = False
    default_keywords["save_embedded_h_core"] = False
    default_keywords["save_embedded_orbitals"] = False
    default_keywords["run_high_level"] = True

    # Checking if the necessary keywords have been defined
    required = ["low_level", "high_level", "basis", "n_active_atoms"]
    if not all([key in keywords for key in required]):
        raise KeyError("You must include keys for each of:\n%s", required)
    # assert 'low_level' in keywords, ('\n Choose level of theory',
    #                                 'for the environment')
    # assert 'high_level' in keywords, ('\n Choose level of theory',
    #                                 'for the active region')
    # assert 'basis' in keywords, '\n Choose a basis set'
    # assert 'n_active_atoms' in keywords, ('\n Provide the number of active',
    #     'atoms, which the first atoms in your coordinates string')

    for key in default_keywords.keys():
        if key not in keywords:
            keywords[key] = default_keywords[key]

    if "n_cl_shell" in keywords and "virtual_projection_basis" not in keywords:
        keywords["virtual_projection_basis"] = keywords["basis"]

    return keywords

keywords = fill_defaults(options)

#run_closed_shell(keywords)

# 1. Low-level whole system calculation

The first step is to run a mean field caluclation of the whole system.    

We also run a high-precision method, Full Configuration Interaction or Coupled Cluster Single Double to determine a reference value.

In [2]:
from pyscf import gto, scf, cc, ao2mo, fci

mol = gto.Mole(atom=keywords['geometry'], basis=keywords['basis'], charge=1, spin=1).build()

ks = scf.UKS(mol)
ks.conv_tol = keywords["e_convergence"]
ks.xc = keywords["low_level"]
e_initial = ks.kernel()

# Store the initial value of h core as this is needed later,
# but is overwritten

initial_h_core = ks.get_hcore()

# Calulate a reference value
mol_copy = copy(mol)
hf = mol_copy.UHF().run()

/usr/local/lib/python3.9/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'


converged SCF energy = -77.2093903345242  <S^2> = 0.75143351  2S+1 = 2.001433
converged SCF energy = -76.7836066237678  <S^2> = 0.75380945  2S+1 = 2.0038058


In [4]:
run_fci = True
if run_fci:
    ref_fci = fci.FCI(hf)
    ref_fci.conv_tol = keywords["e_convergence"]
    fci_result = ref_fci.kernel()

    # This DOES have nuclear energy included!
    expected_energy = fci_result[0]

else:
    ref_cc = cc.CCSD(hf)
    ref_cc.conv_tol = keywords["e_convergence"]
    cc_result = ref_cc.kernel()

    expected_energy = hf.energy_tot() + cc_result[0]

print(f"{e_initial=}, {expected_energy=}")

e_initial=-77.20939033452416, expected_energy=-76.89618023386566


In [5]:
n_occupied_mos_alpha = np.count_nonzero(ks.mo_occ[0] == 1)
n_occupied_mos_beta = np.count_nonzero(ks.mo_occ[1] == 1)

occupied_orbitals_alpha = ks.mo_coeff[0][:, :n_occupied_mos_alpha]
occupied_orbitals_beta = ks.mo_coeff[1][:, :n_occupied_mos_beta]

n_act_aos = mol.aoslice_by_atom()[keywords['n_active_atoms']-1][-1]

# Rotate orbitals so that they're orthogonal
ao_overlap = ks.get_ovlp()
rotated_orbitals_alpha = linalg.fractional_matrix_power(ao_overlap, 0.5) @ occupied_orbitals_alpha
rotated_orbitals_beta = linalg.fractional_matrix_power(ao_overlap, 0.5) @ occupied_orbitals_beta
_alpha, sigma_alpha, right_vectors_alpha = linalg.svd(rotated_orbitals_alpha[:n_act_aos, :])
_beta, sigma_beta, right_vectors_beta = linalg.svd(rotated_orbitals_beta[:n_act_aos, :])

#n_act_mos, n_env_mos = embed.orbital_partition(sigma)
value_diffs_alpha = sigma_alpha[:-1]-sigma_alpha[1:]
value_diffs_beta = sigma_beta[:-1]-sigma_beta[1:]

n_act_mos_alpha = np.argmax(value_diffs_alpha) + 1
n_act_mos_beta = np.argmax(value_diffs_beta) + 1
n_env_mos_alpha = n_occupied_mos_alpha - n_act_mos_alpha
n_env_mos_beta = n_occupied_mos_beta - n_act_mos_beta

# Defining active and environment orbitals and 
act_orbitals_alpha = occupied_orbitals_alpha @ right_vectors_alpha.T[:, :n_act_mos_alpha]
act_orbitals_beta = occupied_orbitals_beta @ right_vectors_beta.T[:, :n_act_mos_beta]
env_orbitals_alpha = occupied_orbitals_alpha @ right_vectors_alpha.T[:, n_act_mos_alpha:n_act_mos_alpha+n_env_mos_alpha]
env_orbitals_beta = occupied_orbitals_beta @ right_vectors_beta.T[:, n_act_mos_beta:n_act_mos_beta+n_env_mos_beta]

act_density_alpha = act_orbitals_alpha @ act_orbitals_alpha.T
act_density_beta = act_orbitals_beta @ act_orbitals_beta.T
env_density_alpha = env_orbitals_alpha @ env_orbitals_alpha.T
env_density_beta = env_orbitals_beta @ env_orbitals_beta.T

print(f"{n_act_mos_alpha=}, {n_act_mos_beta=}, {n_env_mos_alpha=}, {n_env_mos_beta=}")

n_act_mos_alpha=5, n_act_mos_beta=4, n_env_mos_alpha=3, n_env_mos_beta=3


# 3. Calculate the cross subsytem terms

In [6]:
# Retrieving the subsytem energy terms and potential matrices

# The function called looks like this
def open_shell_subsystem(scf, densities):
        #It seems that PySCF lumps J and K in the J array 
        j = ks.get_j(dm = densities)
        two_e_term =  ks.get_veff(dm = densities)

        e_xc = two_e_term.exc
        v_xc = two_e_term - j

        j_total = j[0] + j[1]
        dm_total = densities[0] + densities[1]

        #energy
        e = (
            np.einsum("ij,ji->", ks.get_hcore(), dm_total)
            + 0.5 * (np.einsum("ij,ji->", j_total, dm_total))
            + e_xc
        )

        return e, two_e_term, j, v_xc

dm_active = np.array([act_density_alpha, act_density_beta])
dm_enviro = np.array([env_density_alpha, env_density_beta])
total_dm = act_density_alpha + act_density_beta + env_density_alpha + env_density_beta
two_e_term_total = ks.get_veff(dm=total_dm)
e_xc_total = two_e_term_total.exc

e_act, two_e_act, j_act, v_xc_act  = open_shell_subsystem(mol, dm_active)
e_env, two_e_env, j_env, v_xc_env = open_shell_subsystem(mol, dm_enviro)

j_cross = 0.5 * (
    np.einsum("ij,ij", dm_active[0], j_env[0])
    + np.einsum("ij,ij", dm_enviro[0], j_act[0])
    + np.einsum("ij,ij", dm_active[0], j_env[1])
    + np.einsum("ij,ij", dm_enviro[0], j_act[1])
    + np.einsum("ij,ij", dm_active[1], j_env[1])
    + np.einsum("ij,ij", dm_enviro[1], j_act[1])
    + np.einsum("ij,ij", dm_active[1], j_env[0])
    + np.einsum("ij,ij", dm_enviro[1], j_act[0])
)

# Because of projection we expect kinetic term to be zero
k_cross = 0.0

xc_cross = e_xc_total - two_e_act.exc - two_e_env.exc

# overall two_electron cross energy
two_e_cross = j_cross + k_cross + xc_cross

print(f"{e_act=}, {e_env=}, {two_e_cross=}")

e_act=-71.73108954213181, e_env=-57.912884284654616, two_e_cross=18.95447374759548


# 4. Define $V_{emb}$

We can now define the projector used to orthogonalise the Molecular and Atomic orbitals. From this we calculate the embedding potential.

$P_{\alpha, \beta} = S\gamma^BS$

From this we can now also define the embedding potential.

$V_{emb} = g[\gamma^A, \gamma^B] - g[\gamma^A] + \mu P$

In [7]:
# Define the mu-projector
projector_alpha = keywords['level_shift'] * (ks.get_ovlp() @ env_density_alpha @ ks.get_ovlp())
projector_beta = keywords['level_shift'] * (ks.get_ovlp() @ env_density_beta @ ks.get_ovlp())

v_xc_total = ks.get_veff() - ks.get_j()

# Defining the embedded core Hamiltonian
v_emb_alpha = (j_env[0] + v_xc_total[0] - v_xc_act[0] + projector_alpha)
v_emb_beta = (j_env[1] + v_xc_total[1] - v_xc_act[1] + projector_beta)

print(f"{keywords['level_shift']=}")

keywords['level_shift']=1000000.0


# 5A Run HF of full system with $V_{emb}$ to get $\gamma^A_{emb}$

Here, PsiEmbed gives us the option to stop, outputting values for calculation by other means.

To continue, we run the mean field method, but with the embedding potentials as calulated.

In [35]:
embedded_scf = scf.UHF(mol_copy)
embedded_scf.conv_tol = keywords["e_convergence"]
embedded_scf.mol.nelectron = n_act_mos_alpha + n_act_mos_beta

h_core = ks.get_hcore()

#calculate values for alpha
embedded_scf.get_hcore = lambda *args, **kwargs: h_core + v_emb_alpha

embedded_scf.kernel()

embedded_occ_orbs_alpha = embedded_scf.mo_coeff[0][:, embedded_scf.mo_occ[0]>0]
embedded_density_alpha = embedded_occ_orbs_alpha @ embedded_occ_orbs_alpha.T
vhf1_alpha =embedded_scf.get_veff( dm = embedded_density_alpha)
hcore_alpha = embedded_scf.get_hcore()

#calculate values for beta
embedded_scf.get_hcore = lambda *args, **kwargs: h_core + v_emb_beta

embedded_scf.kernel()

embedded_occ_orbs_beta = embedded_scf.mo_coeff[1][:, embedded_scf.mo_occ[1]>0]
embedded_density_beta = embedded_occ_orbs_beta @ embedded_occ_orbs_beta.T
vhf1_beta =embedded_scf.get_veff( dm = embedded_density_beta)
hcore_beta = embedded_scf.get_hcore()

#Calculate embedding energies
e_emb_alpha = embedded_scf.energy_elec(dm=embedded_density_alpha, h1e = hcore_alpha, vhf=vhf1_alpha)[0]
e_emb_beta = embedded_scf.energy_elec(dm=embedded_density_beta, h1e = hcore_beta, vhf=vhf1_beta)[0]

print(f"{e_emb_alpha=}")
print(f"{e_emb_beta=}")

converged SCF energy = -18.7784860083775  <S^2> = 0.88684521  2S+1 = 2.1324589
SCF not converged.
SCF energy = -18.812320115341 after 50 cycles  <S^2> = 0.84381405  2S+1 = 2.0917113
e_emb_alpha=-33.24597244897784
e_emb_beta=-30.531372238506673


# 6 Calculate correction term

In [45]:
# Compute the correction
# - There are two versions used for different embeddings
dm_correction_alpha = np.einsum("ij,ij", v_emb_alpha, embedded_density_alpha-act_density_alpha)
dm_correction_beta = np.einsum("ij,ij", v_emb_beta, embedded_density_beta-act_density_beta)
dm_correction = dm_correction_alpha + dm_correction_beta

wf_correction_alpha = np.einsum("ij,ij", act_density_alpha, v_emb_alpha)
wf_correction_beta = np.einsum("ij,ij", act_density_beta, v_emb_beta)
wf_correction = wf_correction_alpha + wf_correction_beta

print(f"{wf_correction=}")
print(f"{dm_correction=}")

wf_correction=18.963206252343554
dm_correction=-0.16628592117945118


# 7 Calculate $E[\gamma^A_{emb}]$

## Classical Method - CCSD

In [46]:
from scipy.linalg import LinAlgError

# Can use either of these methods 
# This needs to change if we're not using PySCFEmbed
# The j and k matrices are defined differently in PySCF and Psi4

e_wf_act_alpha = embedded_scf.energy_elec(dm=embedded_density_alpha, h1e = hcore_alpha, vhf=vhf1_alpha)[0]
e_wf_act_beta = embedded_scf.energy_elec(dm=embedded_density_beta, h1e = hcore_beta, vhf=vhf1_beta)[0]
e_act_emb_explicit_alpha = np.einsum("ij,ij", embedded_density_alpha,  initial_h_core + 0.5 * embedded_scf.get_j()[0] - 0.25 * embedded_scf.get_k()[0])
e_act_emb_explicit_beta = np.einsum("ij,ij", embedded_density_beta,  initial_h_core + 0.5 * embedded_scf.get_j()[1] - 0.25 * embedded_scf.get_k()[1])

e_wf_act = e_wf_act_alpha + e_wf_act_beta
e_act_emb_explicit = e_act_emb_explicit_alpha + e_act_emb_explicit_beta


print(f"E_HF alpha = {e_wf_act_alpha}")
print(f"E_HF beta = {e_wf_act_beta}")
print(f"Difference between HF methods: {e_wf_act - e_act_emb_explicit}")
print(f"Difference between HF methods + corrections: {(e_wf_act - wf_correction) - (e_act_emb_explicit + dm_correction)}")

try:
    # Run CCSD as WF method
    ccsd = cc.CCSD(embedded_scf)
    ccsd.conv_tol = keywords["e_convergence"]

    # Set which orbitals are to be frozen
    # The environment orbitals energies have been increased by the projector
    # so they are now at the end of the list, as orbitals are ordered by energy
    shift = mol.nao - n_env_mos_alpha - n_env_mos_beta
    fos = [i for i in range(shift, mol.nao)]
    ccsd.frozen = fos
    ccsd.run()
    correlation = ccsd.e_corr
    e_wf_act += correlation
    
except LinAlgError as e:
    print(e)
    print("Use the HF energy")
    pass

f"{e_wf_act=}"

E_HF alpha = -33.24597244897784
E_HF beta = -30.531372238506673
Difference between HF methods: 18.993945046920402
Difference between HF methods + corrections: 0.19702471575629943
E(UCCSD) = -18.87663635422832  E_corr = -0.06431623888728824


'e_wf_act=-63.8416609263718'

## Quantum Method - VQE

### Set active space

Before setting up the VQE we'll get the active and occupied indices - this will be how we reduce the number of qubits needed.

Active indices include: Occupied MOs assigned to the active region + All unoccupied MOs

In [30]:
from openfermion.ops.representations import InteractionOperator, get_active_space_integrals
from openfermion.linalg import eigenspectrum, expectation
from openfermion.transforms import jordan_wigner

occupied_indices = embedded_scf.mo_occ == 1
occupied_indices_alpha = [i for i, val in enumerate(occupied_indices[0]) if val]
occupied_indices_beta = [i for i, val in enumerate(occupied_indices[1]) if val]

# Try using only active MOs
active_indices_alpha = [i for i in range(len(embedded_scf.mo_occ[0])-n_env_mos_alpha)]
active_indices_beta = [i for i in range(len(embedded_scf.mo_occ[1])-n_env_mos_beta)]

if mos_to_use % 2 == 0:
    active_indices_alpha = active_indices_alpha[:mos_to_use//2]
    active_indices_beta = active_indices_beta[:mos_to_use//2]
else: 
    active_indices_alpha = active_indices_alpha[:mos_to_use//2+1]
    active_indices_beta = active_indices_beta[:mos_to_use//2]

print(f"{len(embedded_scf.mo_occ)}, {embedded_scf.mo_energy}")
print(f"{active_indices_alpha=} \n {occupied_indices_alpha=}")
print(f"{active_indices_beta=} \n {occupied_indices_beta=}")


2, [[-1.14424908e+01 -1.31536528e+00 -1.00976823e+00 -9.98984323e-01
  -6.84886644e-01 -5.84239483e-02  2.57292070e-01  2.79260816e-01
   3.69261219e-01  5.81942331e-01  5.88280593e-01  9.99991435e+05
   9.99999373e+05  9.99999525e+05]
 [-1.14476182e+01 -1.24111704e+00 -9.49221054e-01 -7.01143285e-01
  -4.56098097e-01 -3.20540214e-02  3.34553646e-01  3.36332564e-01
   3.85638854e-01  6.00349875e-01  6.36432559e-01  9.99991442e+05
   9.99999392e+05  9.99999535e+05]]
active_indices_alpha=[0, 1, 2, 3, 4] 
 occupied_indices_alpha=[0, 1, 2, 3, 4]
active_indices_beta=[0, 1, 2, 3, 4] 
 occupied_indices_beta=[0, 1, 2, 3]


### Get Qubit Hamiltonian
Now we'll calculate the one and two body integrals to create a fermionic Hamiltonian.

We then cut out the contribution from the environment orbitals. (Note the reduction in the shape of the integrals!)

In [17]:
from functools import reduce
from pyscf import ao2mo

temp_scf = embedded_scf
temp_mol = mol

# We only want the terms related to the active space
coeff_alpha = temp_scf.mo_coeff[0][:, active_indices_alpha]
coeff_beta = temp_scf.mo_coeff[1][:, active_indices_beta]

n_orbs_alpha = len(active_indices_alpha)
n_orbs_beta = len(active_indices_beta)

# hamiltonian here includes Vemb
one_body_integrals_alpha = coeff_alpha.T @ hcore_alpha @ coeff_alpha
one_body_integrals_beta = coeff_beta.T @ hcore_beta @ coeff_beta

temp_mol.incore_anyway == True

# Get two electron integrals in compressed format.
two_body_compressed_alpha = ao2mo.kernel(temp_mol, coeff_alpha)
two_body_compressed_beta = ao2mo.kernel(temp_mol, coeff_beta)

two_body_integrals_alpha = ao2mo.restore(1, two_body_compressed_alpha, n_orbs_alpha)
two_body_integrals_beta = ao2mo.restore(1, two_body_compressed_beta, n_orbs_beta)

# Openfermion uses pysicist notation whereas pyscf uses chemists
two_body_integrals_alpha = np.asarray(two_body_integrals_alpha.transpose(0, 2, 3, 1), order='C')
two_body_integrals_beta = np.asarray(two_body_integrals_beta.transpose(0, 2, 3, 1), order='C')

print(f"{one_body_integrals_alpha.shape=}, {two_body_integrals_alpha.shape=}")
print(f"{one_body_integrals_beta.shape=}, {two_body_integrals_beta.shape=}")


one_body_integrals_alpha.shape=(5, 5), two_body_integrals_alpha.shape=(5, 5, 5, 5)
one_body_integrals_beta.shape=(5, 5), two_body_integrals_beta.shape=(5, 5, 5, 5)


We have to transform this to spin-orbital coefficients to make a qubit operator in terms of the Pauli basis.

Then we can find the ground state either by diagonalising the Hamiltonian directly, or by running the VQE algorithm.

In [47]:
from openfermion.chem.molecular_data import spinorb_from_spatial
from openfermion.ops.representations import get_active_space_integrals

one_body_coefficients_alpha, two_body_coefficients_alpha = spinorb_from_spatial(one_body_integrals_alpha, two_body_integrals_alpha)
one_body_coefficients_beta, two_body_coefficients_beta = spinorb_from_spatial(one_body_integrals_beta, two_body_integrals_beta)


molecular_hamiltonian_alpha = InteractionOperator(0,
                                            one_body_coefficients_alpha,
                                            0.5 * two_body_coefficients_alpha,)
molecular_hamiltonian_beta = InteractionOperator(0,
                                            one_body_coefficients_beta,
                                            0.5 * two_body_coefficients_beta,)                           

Qubit_Hamiltonian_alpha = jordan_wigner(molecular_hamiltonian_alpha)
Qubit_Hamiltonian_beta = jordan_wigner(molecular_hamiltonian_beta)

e_vqe_act_alpha = eigenspectrum(Qubit_Hamiltonian_alpha)[0]
e_vqe_act_beta = eigenspectrum(Qubit_Hamiltonian_beta)[0]
print(f"{e_vqe_act_alpha=}")
print(f"{e_vqe_act_beta=}")

e_vqe_act_alpha=-46.93022642927296
e_vqe_act_beta=-52.67996032287701


### Add up parts without running VQE

e_act_emb : $\epsilon[\gamma^{A, \{\alpha, \beta\}}_{emb}]$
>energy of the embedded region

e_env : $E[\gamma^B]$ 
>energy of the environment

two_e_cross : $g[\gamma^A, \gamma^B]$
>non-additive two electron term

embed.nre
>The Coulomb energy from nuclear repulsion.

correction : $tr[(\gamma^A_{emb} - \gamma^A)(h^{A in B} - h)]$ (or $tr[\gamma^A(h^{A in B} - h)]$ )
> Correction for embedding

In [41]:
e_nuc = mol.energy_nuc()

e_vqe_emb = e_vqe_act_alpha + e_vqe_act_beta + e_env + two_e_cross + e_nuc - wf_correction_alpha - wf_correction_beta
e_wf_emb = e_wf_act_alpha + e_wf_act_beta + e_env + two_e_cross + e_nuc - wf_correction_alpha -  wf_correction_beta
print("Component contributions")
print(f"{e_vqe_act_alpha=}, {e_vqe_act_beta=}")
print(f"{e_env=}, {two_e_cross=}, {wf_correction=}, {dm_correction=}, {e_nuc=}")
print(f"{occupied_indices=}, {active_indices_alpha=}, Total Usable MOs={len(ks.mo_occ)-n_env_mos_alpha}\n")
# Print out the final value.
print(f"FCI Energy:\t\t{expected_energy:.4f}")

print(f"Used MOs={len(active_indices)}")
print(f"Act MOs={n_act_mos}")
print(f"Env MOs={n_env_mos}")
print(f"Total MOs={len(ks.mo_occ)}\n")

print(f"DFT Energy:\t\t{e_initial:.4f}")
print(f"Error:\t\t\t{(expected_energy-e_initial)*100/expected_energy:.2f}%")

print(f"WF Embedding Energy:\t{e_wf_emb:.4f}")
print(f"Error:\t\t\t{(expected_energy-e_wf_emb)*100/expected_energy:.2f}%")
print(f"VQE Embedding Energy:\t{e_vqe_emb:.4f}")
print(f"Error:\t\t\t{(expected_energy-e_vqe_emb)*100/expected_energy:.2f}%")

Component contributions
e_vqe_act_alpha=-46.93022642927296, e_vqe_act_beta=-52.67996032287701
e_env=-57.912884284654616, two_e_cross=18.95447374759548, wf_correction_alpha=10.861361895055865, dm_correction_alpha=-0.7400637021676175, wf_correction_beta=8.101844357287689, dm_correction_beta=0.5737777809881663, e_nuc=33.51898302939866
occupied_indices=array([[ True,  True,  True,  True,  True, False, False, False, False,
        False, False, False, False, False],
       [ True,  True,  True,  True, False, False, False, False, False,
        False, False, False, False, False]]), active_indices_alpha=[0, 1, 2, 3, 4], Total Usable MOs=-1

FCI Energy:		-76.8962
DFT Energy:		-77.2094
Error:			-0.41%
WF Embedding Energy:	-88.1800
Error:			-14.67%
VQE Embedding Energy:	-124.0128
Error:			-61.27%


### Pennylane VQE

Taking the qubit operator we defined with open fermion, we can perform VQE using the CCSD ansatz.

In [18]:
import pennylane as qml
from pennylane import qchem
from pennylane.templates import UCCSD, ParticleConservingU2
from functools import partial

h_alpha = qchem.convert_observable(Qubit_Hamiltonian_alpha)
h_beta = qchem.convert_observable(Qubit_Hamiltonian_beta)

qubits1 = len(h_alpha.wires)
qubits2 = len(h_beta.wires)
print(f"{qubits1=}")
print(f"{qubits2=}")

electrons = n_act_mos*2

ref_state = qchem.hf_state(electrons, qubits)

# Generate single and double excitations
singles, doubles = qchem.excitations(electrons, qubits)

# Map excitations to the wires the UCCSD circuit will act on
s_wires, d_wires = qchem.excitations_to_wires(singles, doubles)

# Define the device
#dev = qml.device('default.qubit', wires=qubits)
dev = qml.device('default.qubit', wires=qubits)


# Define the UCCSD ansatz
#ansatz = partial(UCCSD, init_state=ref_state, s_wires=s_wires, d_wires=d_wires)
ansatz = partial(ParticleConservingU2, init_state=ref_state)

# Define the cost function
cost_fn = qml.ExpvalCost(ansatz, h, dev)

opt = qml.GradientDescentOptimizer(stepsize=1)

# Compute the expectation value of 'h' for given set of parameters 'params'
layers = 1


qubits1=10
qubits2=10


NameError: name 'n_act_mos' is not defined

In [ ]:
#UCCSD Params
#params = np.random.normal(0, np.pi, len(singles) + len(doubles))

params = qml.init.particle_conserving_u2_normal(layers, qubits)
explicit_e_vqe_act = None

if run_vqe:
    max_iterations = 100
    conv_tol = keywords["e_convergence"]
    energy = [cost_fn(params)]
    param_vals = [params]

    print("Beginning Iterations")
    print(f"Step = 0,  Energy = {energy[-1]:.8f} Ha")
    for n in range(max_iterations):
        params, prev_energy = opt.step_and_cost(cost_fn, params)

        energy.append(cost_fn(params))
        param_vals.append(params)

        conv = np.abs(energy[-1] - prev_energy)

        # if n % 2 == 0:
        print(f"Step = {n+1},  Energy = {energy[-1]:.8f} Ha")

        if conv <= conv_tol and conv != 0:
            break

    explicit_e_vqe_act = energy[-1]

    print("\n" f"Final value of the ground-state energy = {explicit_e_vqe_act:.8f} Ha")
    print("\n" f"Optimal value of the circuit parameter = {param_vals[-1]}")

# 8. Add all the parts up with VQE

e_act_emb : $\epsilon[\gamma^A_{emb}]$
>energy of the embedded region

e_env : $E[\gamma^B]$ 
>energy of the environment

two_e_cross : $g[\gamma^A, \gamma^B]$
>non-additive two electron term

embed.nre
>The Coulomb energy from nuclear repulsion.

correction : $tr[(\gamma^A_{emb} - \gamma^A)(h^{A in B} - h)]$ (or $tr[\gamma^A(h^{A in B} - h)]$ )
> Correction for embedding

In [ ]:
e_nuc = mol.energy_nuc()

e_vqe_emb = e_vqe_act + e_env + two_e_cross + e_nuc - wf_correction 
e_wf_emb = e_wf_act + e_env + two_e_cross + e_nuc - wf_correction 
print("Component contributions")
print(f"{e_vqe_act=}, {e_wf_act=}")
print(f"{e_env=}, {two_e_cross=}, {wf_correction=}, {dm_correction=}, {e_nuc=}\n")
# Print out the final value.
print(f"FCI Energy:\t\t\t{expected_energy:.4f}")

print(f"Used MOs={len(active_indices)}")
print(f"Act MOs={n_act_mos}")
print(f"Env MOs={n_env_mos}")
print(f"Total MOs={len(ks.mo_occ)}\n")

print(f"DFT Energy:\t\t\t{e_initial:.4f}")
print(f"Error:\t\t\t\t{(expected_energy-e_initial)*100/expected_energy:.2f}%\n")

print(f"WF Embedding Energy:\t\t{e_wf_emb:.4f}")
print(f"Error:\t\t\t\t{(expected_energy-e_wf_emb)*100/expected_energy:.2f}%\n")

print(f"VQE Embedding Energy:\t\t{e_vqe_emb:.4f}")
print(f"Error:\t\t\t\t{(expected_energy-e_vqe_emb)*100/expected_energy:.2f}%\n")

if explicit_e_vqe_act:
    explicit_e_vqe_emb = explicit_e_vqe_act + e_env + two_e_cross + e_nuc - wf_correction 
    print(f"Explicit VQE Embedding Energy:\t{explicit_e_vqe_emb:.4f}")
    print(f"Error:\t\t\t\t{(expected_energy-explicit_e_vqe_emb)*100/expected_energy:.2f}%")


Component contributions
e_vqe_act=-61.69158831931323, e_wf_act=-61.75537237033033
e_env=-21.7637520735885, two_e_cross=15.553595561079673, wf_correction=15.566037482763932, dm_correction=0.45640623149108706, e_nuc=9.285714221677825

FCI Energy:			-76.2060
Used MOs=5
Act MOs=3
Env MOs=2
Total MOs=18

DFT Energy:			-67.9422
Error:				10.84%

WF Embedding Energy:		-74.2459
Error:				2.57%

VQE Embedding Energy:		-74.1821
Error:				2.66%

